In [3]:
import os

In [4]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs'

## config_entity.py

In [7]:
# Updating the Entity 
# The same variables from the config.yaml is here like root_dir, source_url, local_data_file, unzip_dir
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  
    root_dir: Path
    source_URL: str # the written type is string
    local_data_file: Path 
    unzip_dir: Path 

## configuration.py

In [8]:
# Updating configuration manager in src config
# Next we need to read the Yaml file.then we will write some functions in the constants folder. we define that 3 functionto read the yaml files.
# after that we will import functions like read_yaml, create_directories from utils.common

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,  # these variables like CONFIG_FILE_PATH has come from the constants folder
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepaht = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepaht)

        create_directories([self.config.artifacts_root]) # here we are creating th artifacts folder | the artifacts_root has come from config.yaml

    def get_data_ingestion_config(self) -> DataIngestionConfig:  # here we have defined our written type
        config = self.config.data_ingestion

        create_directories([config.root_dir])   # the root_dir has come from config.yaml

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

## components/data_ingestion.py

In [9]:
# Updating the components

import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
            url = self.config.source_URL, # the surce_url is mentioned in the comfig.yaml
            filename = self.config.local_data_file
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f'file already exists of size: {get_size(path(self.config.local_data_file))}')

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## pipeline/stage_01_data_ingestion.py

In [10]:
# Update the Pipeline

try:
    config = ConfigurationManager() # configuration manager
    data_ingestion_config = config.get_data_ingestion_config() # configuration manager
    data_ingestion = DataIngestion(config=data_ingestion_config) # components
    data_ingestion.download_file() # components
    data_ingestion.extract_zip_file() # components
except Exception as e:
    raise e

[2025-07-12 15:51:29,387: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-12 15:51:29,390: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-12 15:51:29,392: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-12 15:51:29,393: INFO: common: created directory at: artifacts]
[2025-07-12 15:51:29,394: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-12 15:51:30,067: INFO: 2413313259: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 17622
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e8867d7a51c87b7e14e8503577a8caa110675b63d196d73096bb4fe655fb7aec"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 27E2:2D4348:A112CD:BCA563:68725C09
Accept-Ranges: bytes
Date: Sat, 12